In [1]:
!git clone https://github.com/Acciorocketships/sae.git

Cloning into 'sae'...
remote: Enumerating objects: 662, done.
remote: Counting objects: 100% (662/662), done.
remote: Compressing objects: 100% (279/279), done.
remote: Total 662 (delta 392), reused 641 (delta 376), pack-reused 0 (from 0)
Receiving objects: 100% (662/662), 20.79 MiB | 22.13 MiB/s, done.
Resolving deltas: 100% (392/392), done.


In [2]:
%cd sae

/content/sae


In [3]:
!pip install -e


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

-e option requires 1 argument


In [262]:
import torch
from torch import nn, optim


In [263]:
from sae import AutoEncoder

In [264]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split


In [265]:
class GeneInteractionNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(GeneInteractionNN, self).__init__()
        self.hidden = nn.Linear(input_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = F.relu(self.hidden(x))
        x = torch.sigmoid(self.output(x))
        return x

In [266]:
#Autoencoder being trained for each gene across all individuals (each batch is a different ind)
import torch
from sae import AutoEncoder
from sklearn.preprocessing import StandardScaler


num_individuals = 30
num_genes = 15         #Number of genes per individual
max_n = 200           #Max input dimension for each gene (discuss later)
embedding_dim = 4     #latent space dimension
epochs = 100

# 1. Generating synthetic data in which each class has a different distribution
group_0 = torch.normal(mean=1, std=0.4, size=(15, num_genes, max_n))
group_1 = torch.normal(mean=1.5, std=0.3, size=(15, num_genes, max_n))

data = torch.cat((group_0, group_1), dim=0)
labels = torch.cat((torch.zeros(15), torch.ones(15)))


data_scaled = torch.empty_like(data)
scaler = StandardScaler()
for gene_idx in range(num_genes):
    gene_data = data[:, gene_idx, :]  # Shape: (30, max_n)
    data_scaled[:, gene_idx, :] = torch.tensor(
        scaler.fit_transform(gene_data), dtype=torch.float32
    )

#place holder for final embeddings: (num_individuals, num_genes * embedding_dim)
final_embeddings = torch.empty((num_individuals, num_genes * embedding_dim))

# 2. Training AutoEncoder for each gene
for gene_idx in range(num_genes):
    print(f"Training autoencoder for Gene {gene_idx + 1}/{num_genes}")

    gene_data = data_scaled[:, gene_idx, :]

    model = AutoEncoder(dim=max_n, hidden_dim=embedding_dim, max_n=max_n)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    batch = torch.arange(0, num_individuals)

    #Training autoencoder
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        #forward
        _, batchr = model(gene_data, batch)
        loss = model.loss()["loss"]

        #backward pass and optimization
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

    #extracting latent embeddings for the current gene
    model.eval()
    with torch.no_grad():
        z = model.encoder(gene_data, batch)  #shape: (num_individuals, embedding_dim)


    final_embeddings[:, gene_idx * embedding_dim:(gene_idx + 1) * embedding_dim] = z


print("Final Embeddings Shape:", final_embeddings.shape)


Training autoencoder for Gene 1/15
Epoch 10/100, Loss: 0.3960404694080353


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/content/sae/sae/sae_model.py:52: UserWarning: var(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/Reduc

Epoch 20/100, Loss: 0.35335826873779297
Epoch 30/100, Loss: 0.31316134333610535
Epoch 40/100, Loss: 0.27566424012184143
Epoch 50/100, Loss: 100.77806091308594
Epoch 60/100, Loss: 79.06385803222656
Epoch 70/100, Loss: 66.37223052978516
Epoch 80/100, Loss: 63.674415588378906
Epoch 90/100, Loss: 62.52069854736328
Epoch 100/100, Loss: 61.6512565612793
Training autoencoder for Gene 2/15
Epoch 10/100, Loss: 0.30938199162483215
Epoch 20/100, Loss: 0.27877411246299744
Epoch 30/100, Loss: 108.59234619140625
Epoch 40/100, Loss: 100.84932708740234
Epoch 50/100, Loss: 85.40225219726562
Epoch 60/100, Loss: 66.30450439453125
Epoch 70/100, Loss: 63.19626235961914
Epoch 80/100, Loss: 61.25282287597656
Epoch 90/100, Loss: 58.87060546875
Epoch 100/100, Loss: 58.77933883666992
Training autoencoder for Gene 3/15
Epoch 10/100, Loss: 114.65299224853516
Epoch 20/100, Loss: 90.00298309326172
Epoch 30/100, Loss: 82.51826477050781
Epoch 40/100, Loss: 78.42537689208984
Epoch 50/100, Loss: 71.60430145263672
Epoch

In [267]:
x_train, x_test, y_train, y_test = train_test_split(final_embeddings.detach().numpy(), labels.numpy(), test_size=0.2, random_state=42)

In [268]:
x_train = torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [269]:
#The MLP part
input_dim = final_embeddings.shape[1]
hidden_dim = 12
model = GeneInteractionNN(input_dim, hidden_dim)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(x_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

Epoch 10/100, Loss: 0.46882691979408264
Epoch 20/100, Loss: 0.29003143310546875
Epoch 30/100, Loss: 0.19289427995681763
Epoch 40/100, Loss: 0.1378091722726822
Epoch 50/100, Loss: 0.10305505245923996
Epoch 60/100, Loss: 0.07706446200609207
Epoch 70/100, Loss: 0.057174187153577805
Epoch 80/100, Loss: 0.04312387481331825
Epoch 90/100, Loss: 0.03346600756049156
Epoch 100/100, Loss: 0.02675621025264263


In [270]:
#Testing
model.eval()
with torch.no_grad():
    y_pred_test = model(x_test)
    test_loss = criterion(y_pred_test, y_test)
    print(f"Test Loss: {test_loss.item()}")

Test Loss: 0.022545374929904938


In [271]:
#Rounding probabilities into 0 and 1 to compute accuracy, precision
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

predicted_classes = (y_pred_test > 0.5).int()

y_true = y_test.squeeze().numpy()
y_pred = predicted_classes.squeeze().numpy()

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Accuracy: 1.00
Precision: 1.00
Recall: 1.00
F1 Score: 1.00


In [272]:
y_pred_test

tensor([[0.9806],
        [0.9946],
        [0.9910],
        [0.9942],
        [0.0293],
        [0.0635]])

In [110]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scanpy

In [113]:
#To read the synthetic data
import scanpy as sc
adata = sc.read_h5ad("/content/drive/MyDrive/Colab Notebooks/processed_dataset_16genes.h5ad")

In [123]:
import pandas as pd
phenotype_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/phenotypes_16gene.csv')

In [124]:
phenotype_df.rename(columns={'Sample_ID': 'subject_id'}, inplace=True)

In [193]:
adata.obs['subject_id'] = adata.obs['subject_id'].astype(int)

In [194]:
adata.obs = adata.obs.merge(phenotype_df, on='subject_id', how='left')

In [196]:
unique_subjects = adata.obs.groupby('subject_id')['binary_y'].unique()

In [197]:
for subject_id, binary_y_values in unique_subjects.items(): print(f'Subject ID: {subject_id}, Binary Y: {binary_y_values}')

Subject ID: 1, Binary Y: [1]
Subject ID: 2, Binary Y: [0]
Subject ID: 3, Binary Y: [1]
Subject ID: 4, Binary Y: [0]
Subject ID: 5, Binary Y: [0]
Subject ID: 6, Binary Y: [0]
Subject ID: 7, Binary Y: [0]
Subject ID: 8, Binary Y: [1]
Subject ID: 9, Binary Y: [0]
Subject ID: 10, Binary Y: [0]
Subject ID: 11, Binary Y: [0]
Subject ID: 12, Binary Y: [0]
Subject ID: 13, Binary Y: [0]
Subject ID: 14, Binary Y: [1]
Subject ID: 15, Binary Y: [0]
Subject ID: 16, Binary Y: [1]
Subject ID: 17, Binary Y: [0]
Subject ID: 18, Binary Y: [0]
Subject ID: 19, Binary Y: [0]
Subject ID: 20, Binary Y: [0]


In [296]:
#To perform on the adata above

embedding_dim = 4  #latent space dimension
epochs = 100
n_max = 500        #maximum number of cells per gene for each individual

# 1.Extracting expression data and metadata
expression_data = adata.X.toarray()
subject_ids = adata.obs["subject_id"].values
phenotypes = adata.obs["binary_y"].values


n_individuals = len(set(subject_ids))
n_genes = expression_data.shape[1]

#To ensure correct phenotype order
labels = []
for individual_id in sorted(set(subject_ids)):
    labels.append(np.unique(phenotypes[adata.obs["subject_id"] == individual_id])[0])
labels = torch.tensor(labels, dtype=torch.float32)


#Placeholder for final embeddings: (n_individuals, n_genes * embedding_dim)
final_embeddings = torch.empty((n_individuals, n_genes * embedding_dim))



# 2. Processing each gene
for gene_idx in range(n_genes):
    print(f"Processing Gene {gene_idx + 1}/{n_genes}")
    current_gene_data = []

    for individual_id in sorted(set(subject_ids)):
        #Extracting cells for the current gene and individual
        individual_cells = expression_data[subject_ids == individual_id, gene_idx]


        if len(individual_cells) > n_max:
            individual_cells = individual_cells[:n_max]

        #To pad with zeros if fewer than n_max (?????????????should I do it? since I think it is okay for sets to have different length)
        padded_cells = torch.zeros(n_max)
        padded_cells[:len(individual_cells)] = torch.tensor(individual_cells, dtype=torch.float32)


        current_gene_data.append(padded_cells.unsqueeze(0))

    #to stack all individuals' data for the current gene
    current_gene_data = torch.cat(current_gene_data, dim=0)  # shape: (n_individuals, n_max)

    #Scaling for the current gene (should be like this or all scaled before????????????????)
    scaler = StandardScaler()
    current_gene_data = torch.tensor(scaler.fit_transform(current_gene_data), dtype=torch.float32)

    #initialization of autoencoder for the current gene
    model = AutoEncoder(dim=n_max, hidden_dim=embedding_dim, max_n=n_max)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    #batch indices for individuals
    batch = torch.arange(0, n_individuals)

    #Training
    torch.manual_seed(641555)
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        #forard
        _, batchr = model(current_gene_data, batch)
        loss = model.loss()["loss"]

        #backward
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

    #to extract latent embeddings for the current gene
    model.eval()
    with torch.no_grad():
        z = model.encoder(current_gene_data, batch)  # Shape: (n_individuals, embedding_dim)

    final_embeddings[:, gene_idx * embedding_dim:(gene_idx + 1) * embedding_dim] = z


print("Final Embeddings Shape:", final_embeddings.shape)


Processing Gene 1/16


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Epoch 10/100, Loss: 2.136932134628296
Epoch 20/100, Loss: 1.9801450967788696
Epoch 30/100, Loss: 1.6293604373931885
Epoch 40/100, Loss: 1.3690032958984375
Epoch 50/100, Loss: 1.09722101688385
Epoch 60/100, Loss: 0.9792564511299133
Epoch 70/100, Loss: 0.8703702688217163
Epoch 80/100, Loss: 0.6350077390670776
Epoch 90/100, Loss: 0.44168153405189514
Epoch 100/100, Loss: 0.26999136805534363
Processing Gene 2/16
Epoch 10/100, Loss: 1.7738698720932007


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/content/sae/sae/sae_model.py:52: UserWarning: var(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/Reduc

Epoch 20/100, Loss: 1.7124788761138916
Epoch 30/100, Loss: 1.6508716344833374
Epoch 40/100, Loss: 1.5894529819488525
Epoch 50/100, Loss: 1.5284219980239868
Epoch 60/100, Loss: 1.4678926467895508
Epoch 70/100, Loss: 1.4079617261886597
Epoch 80/100, Loss: 1.3487228155136108
Epoch 90/100, Loss: 1.2902724742889404
Epoch 100/100, Loss: 1.2327101230621338
Processing Gene 3/16
Epoch 10/100, Loss: 1.6786295175552368


...

Epoch 20/100, Loss: 1.6015760898590088
Epoch 30/100, Loss: 1.5332634449005127
Epoch 40/100, Loss: 1.4654967784881592
Epoch 50/100, Loss: 1.398390293121338
Epoch 60/100, Loss: 1.332056999206543
Epoch 70/100, Loss: 1.2666032314300537
Epoch 80/100, Loss: 1.202135682106018
Epoch 90/100, Loss: 1.1387571096420288
Epoch 100/100, Loss: 1.0765697956085205
Processing Gene 15/16
Epoch 10/100, Loss: 1.670692801475525


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/content/sae/sae/sae_model.py:52: UserWarning: var(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/Reduc

Epoch 20/100, Loss: 1.6014978885650635
Epoch 30/100, Loss: 1.5327601432800293
Epoch 40/100, Loss: 1.4646713733673096
Epoch 50/100, Loss: 1.3972933292388916
Epoch 60/100, Loss: 1.3307337760925293
Epoch 70/100, Loss: 1.2650907039642334
Epoch 80/100, Loss: 1.2004634141921997
Epoch 90/100, Loss: 1.1369515657424927
Epoch 100/100, Loss: 1.0746541023254395
Processing Gene 16/16
Epoch 10/100, Loss: 1.6703145503997803


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/content/sae/sae/sae_model.py:52: UserWarning: var(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/Reduc

Epoch 20/100, Loss: 1.6015806198120117
Epoch 30/100, Loss: 1.5332773923873901
Epoch 40/100, Loss: 1.4655174016952515
Epoch 50/100, Loss: 1.3984166383743286
Epoch 60/100, Loss: 1.332087516784668
Epoch 70/100, Loss: 1.266637921333313
Epoch 80/100, Loss: 1.2021733522415161
Epoch 90/100, Loss: 1.1387979984283447
Epoch 100/100, Loss: 1.0766133069992065
Final Embeddings Shape: torch.Size([20, 64])


In [301]:
#Train and testing preparation
x_train, x_test, y_train, y_test = train_test_split(final_embeddings.detach().numpy(), labels.numpy(), test_size=0.2, random_state=1055)

x_train = torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [302]:
#The MLP part
input_dim = final_embeddings.shape[1]
hidden_dim = 12
model = GeneInteractionNN(input_dim, hidden_dim)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(x_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

Epoch 10/100, Loss: 0.5285037159919739
Epoch 20/100, Loss: 0.4578450322151184
Epoch 30/100, Loss: 0.3982967734336853
Epoch 40/100, Loss: 0.3443358540534973
Epoch 50/100, Loss: 0.29556071758270264
Epoch 60/100, Loss: 0.25395771861076355
Epoch 70/100, Loss: 0.21773545444011688
Epoch 80/100, Loss: 0.18595059216022491
Epoch 90/100, Loss: 0.15882326662540436
Epoch 100/100, Loss: 0.13613905012607574


In [303]:
#Testing
model.eval()
with torch.no_grad():
    y_pred_test = model(x_test)
    test_loss = criterion(y_pred_test, y_test)
    print(f"Test Loss: {test_loss.item()}")

Test Loss: 0.12266811728477478


In [305]:
#Rounding probabilities into 0 and 1 to compute accuracy, precision
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

predicted_classes = (y_pred_test > 0.5).int()


y_true = y_test.squeeze().numpy()
y_pred = predicted_classes.squeeze().numpy()


accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)


print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Accuracy: 1.00
Precision: 1.00
Recall: 1.00
F1 Score: 1.00
